pandas partition handling
----

파이썬에서 판다스에서 sql window 함수의 기능을 이용해보자
- 참고 문서 : https://rfriend.tistory.com/493

In [1]:
import pandas as pd

In [2]:
file_path = 'Stock_Dataset(2017_07_06)/000020.csv'
df = pd.read_csv(file_path)

In [4]:
df.head()

,Date,Close,Open,High,Low,Volume,Code,Company,Up&Down,Rate
0,2005-04-27,11250,10750,11500,10650,31610,20,동화약품,0.0,0.000000
1,2005-04-28,11050,11200,11450,10950,12670,20,동화약품,-200.0,-1.777778
2,2005-04-29,10900,10550,10950,10550,15280,20,동화약품,-150.0,-1.357466
3,2005-05-02,11200,10900,11300,10750,71860,20,동화약품,300.0,2.752294
4,2005-05-03,11700,11450,12150,11400,150848,20,동화약품,500.0,4.464286


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025 entries, 0 to 3024
Data columns (total 10 columns):
Date       3025 non-null object
Close      3025 non-null int64
Open       3025 non-null int64
High       3025 non-null int64
Low        3025 non-null int64
Volume     3025 non-null int64
Code       3025 non-null int64
Company    3025 non-null object
Up&Down    3025 non-null float64
Rate       3025 non-null float64
dtypes: float64(2), int64(6), object(2)
memory usage: 236.4+ KB


오늘 시초가와 어제 시초가를 나란히 놓도록 하겠다

In [7]:
df['yesterday'] = df['Open'].shift(1)

In [9]:
df[['Open', 'yesterday']].head()

,Open,yesterday
0,10750,NaN
1,11200,10750.0
2,10550,11200.0
3,10900,10550.0
4,11450,10900.0


In [28]:
df['yesterday'] = df['Open'].shift(-1)

In [30]:
df.head()

,Date,Close,Open,High,Low,Volume,Code,Company,Up&Down,Rate,yesterday
0,2005-04-27,11250,10750,11500,10650,31610,20,동화약품,0.0,0.000000,11200.0
1,2005-04-28,11050,11200,11450,10950,12670,20,동화약품,-200.0,-1.777778,10550.0
2,2005-04-29,10900,10550,10950,10550,15280,20,동화약품,-150.0,-1.357466,10900.0
3,2005-05-02,11200,10900,11300,10750,71860,20,동화약품,300.0,2.752294,11450.0
4,2005-05-03,11700,11450,12150,11400,150848,20,동화약품,500.0,4.464286,13450.0


In [31]:
df.tail()

,Date,Close,Open,High,Low,Volume,Code,Company,Up&Down,Rate,yesterday
3020,2017-06-30,8470,8450,8540,8410,48558,20,동화약품,60.0,0.713436,8470.0
3021,2017-07-03,8440,8470,8510,8410,25073,20,동화약품,-30.0,-0.354191,8450.0
3022,2017-07-04,8430,8450,8490,8370,44152,20,동화약품,-10.0,-0.118483,8430.0
3023,2017-07-05,8340,8430,8440,8300,62841,20,동화약품,-90.0,-1.067616,8340.0
3024,2017-07-06,8400,8340,8420,8300,48957,20,동화약품,60.0,0.719424,NaN


이렇게 간단하게 옮길 수 있다.

위의 링크에서는 조금 더 복잡하고 다양한 경우를 대응할 수 있다.

In [32]:
ts = pd.DataFrame(
    {'dt': ['2019-12-01', '2019-12-02', '2019-12-03', '2019-12-04', 
            '2019-12-01', '2019-12-02', '2019-12-03', '2019-12-04'], 
     'id': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'], 
     'val': [5, 6, 7, 8, 13, 14, 15, 16]
    }
)
ts

,dt,id,val
0,2019-12-01,a,5
1,2019-12-02,a,6
2,2019-12-03,a,7
3,2019-12-04,a,8
4,2019-12-01,b,13
5,2019-12-02,b,14
6,2019-12-03,b,15
7,2019-12-04,b,16


In [33]:
# sql에서 lag 역할을 하는 것
ts['val_lag_1'] =  ts.sort_values(by='dt', ascending=True).groupby('id')['val'].shift(1)

In [34]:
ts

,dt,id,val,val_lag_1
0,2019-12-01,a,5,NaN
1,2019-12-02,a,6,5.0
2,2019-12-03,a,7,6.0
3,2019-12-04,a,8,7.0
4,2019-12-01,b,13,NaN
5,2019-12-02,b,14,13.0
6,2019-12-03,b,15,14.0
7,2019-12-04,b,16,15.0


In [35]:
# sql에서 lead 역할을 하는 것

ts['val_lead_1'] = ts.sort_values(by='dt', ascending=False).groupby('id')['val'].shift(1)

In [36]:
ts

,dt,id,val,val_lag_1,val_lead_1
0,2019-12-01,a,5,NaN,6.0
1,2019-12-02,a,6,5.0,7.0
2,2019-12-03,a,7,6.0,8.0
3,2019-12-04,a,8,7.0,NaN
4,2019-12-01,b,13,NaN,14.0
5,2019-12-02,b,14,13.0,15.0
6,2019-12-03,b,15,14.0,16.0
7,2019-12-04,b,16,15.0,NaN
